In [1]:
from wrappers import graph_from_point_wrapper
import ipywidgets as widgets
from IPython.display import display, Image
import time

In [2]:
# Debouncing shit
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
        return debounced
    return decorator

In [3]:
def read_and_return_img():
    file = open("images/test.jpg", "rb")
    image = file.read()
    return image

In [4]:
def format_second_caption():
    return 'Image below: map with {radius} meter radius; {fig_size}in. figure size; {color}'.format(
        radius=radius_slider.value,
        fig_size=fig_slider.value,
        color=color_picker.value) 

In [5]:
@debounce(1.0)
def shared_event_handler(change):
    second_caption.value = format_second_caption()
    graph_from_point_wrapper(
        point=(lat_text.value, lon_text.value),
        distance=radius_slider.value,
        fig_height=fig_slider.value,
        network_type='drive',
        distance_type='bbox',
        color=color_picker.value,
        save=True
    )
    image = read_and_return_img()
    img_display.value = image


In [6]:
first_caption = widgets.Label(value='~~~~~ Change options below to produce new map ~~~~~')

lat_text = widgets.FloatText(
    value=36.005800,
    placeholder='Type latitude',
    description='Latitude:',
    disabled=False
)

lon_text = widgets.FloatText(
    value=-78.897020,
    placeholder='Type longitude',
    description='Longitude:',
    disabled=False
)
radius_slider = widgets.IntSlider(
    min=500,
    max=5000,
    step=1000,
    description='Map radius:',
    value=1000
)
fig_slider = widgets.IntSlider(
    min=8,
    max=24,
    step=2,
    description='Figure size:',
    value=8
)

color_picker = widgets.ColorPicker(
    concise=False,
    description='Pick a color:',
    value='darkolivegreen',
    disabled=False
)

img_display = widgets.Image(
    value=read_and_return_img()
)

second_caption = widgets.Label(value=format_second_caption())

lat_text.observe(shared_event_handler, names='value')
lon_text.observe(shared_event_handler, names='value')
radius_slider.observe(shared_event_handler, names='value')
fig_slider.observe(shared_event_handler, names='value')
color_picker.observe(shared_event_handler, names='value')

### Nice resource to help find coordinates: https://www.latlong.net/convert-address-to-lat-long.html

In [7]:
display(first_caption, lat_text, lon_text, radius_slider, fig_slider, color_picker, second_caption, img_display)

Label(value='~~~~~ Change options below to produce new map ~~~~~')

FloatText(value=36.0058, description='Latitude:')

FloatText(value=-78.89702, description='Longitude:')

IntSlider(value=1000, description='Map radius:', max=5000, min=500, step=1000)

IntSlider(value=8, description='Figure size:', max=24, min=8, step=2)

ColorPicker(value='darkolivegreen', description='Pick a color:')

Label(value='Image below: map with 1000 meter radius; 8in. figure size; darkolivegreen')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\…